# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [2]:
#Your code here
# pair groups with actions
# compare actions
import csv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import seaborn as sns
%matplotlib inline

df = pd.read_csv('homepage_actions.csv', index_col=0)
df.count()

id        8188
group     8188
action    8188
dtype: int64

In [3]:
clicks = set(df[df.action=='click']['id'].unique())
views = set(df[df.action=='view']['id'].unique())
x=df[df.action=='view']['id'].unique()
len(clicks)+len(views)

8188

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [4]:
#Your code here
df['count'] = 1
control = df[df.group=='control'].pivot(index='id', columns='action',values='count')
control = control.fillna(value=0)
experiment = df[df.group=='experiment'].pivot(index='id', columns='action', values='count')
experiment = experiment.fillna(value=0)
experiment

action,click,view
id,,
182988,0.0,1.0
183136,0.0,1.0
183141,1.0,1.0
183283,0.0,1.0
183389,0.0,1.0
...,...,...
935382,0.0,1.0
935576,0.0,1.0
935742,1.0,1.0


In [8]:
control_percentage= control['click'].sum()/control['view'].sum()
experimental_percentage=experiment['click'].sum()/experiment['view'].sum()
diff_in_clicks=experimental_percentage-control_percentage
print(diff_in_clicks)

0.030034443684015644


In [111]:
def welch_t(x, y):
    numerator = x.mean() - y.mean()
    denominator = np.sqrt(x.var(ddof=1)/x.size + y.var(ddof=1)/y.size)
    return np.abs(numerator/denominator)
welch_t(control['click'], experiment['click'])

2.615440020788212

In [9]:
def welch_df(x, y):
    
    
    s1 = x.var(ddof=1) 
    s2 = y.var(ddof=1)
    n1 = x.size
    n2 = y.size
    
    numerator = (s1/n1 + s2/n2)**2
    denominator = (s1/ n1)**2/(n1 - 1) + (s2/ n2)**2/(n2 - 1)
    
    return numerator/denominator

welch_df(control['click'], experiment['click'])


6211.149291520598

In [120]:
t =welch_t(control['click'], experiment['click'])
degrees =welch_df(control['click'], experiment['click'])


In [124]:
p = 1 - stats.t.cdf(t, degrees)
print(p)

0.004466402814337078


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [126]:
#Your code here
expected_click_exp=control.click.mean()
projected_experiment_clicks = expected_click_exp * len(experiment)
print(projected_experiment_clicks)

838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [127]:
#Your code here
n = len(experiment)
p = expected_click_exp
var = n * p * (1-p)
std = np.sqrt(var)
print(std)

24.568547907005815


In [ ]:
z_score = (experiment.click.sum()-projected_experiment_clicks)/std
print(z_score)

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [135]:
#Your code here
p_val = stats.norm.sf(z_score)
p_val

0.00012486528006951198

### Analysis:

Does this result roughly match that of the previous statistical test?

Yes, the p-value is slightly lower but it certainly meets the threshold. Therefore, we can say with a high degree of confidence that the design seems to be more effective.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.